In [396]:
import os
from datetime import datetime

import numpy as np
import pandas as pd

os.chdir(os.getcwd() + "\\..\\data\\")

In [446]:
months = ["January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December"]

def parse_measure_data(data : pd.DataFrame):
    # Remove header
    data.drop(range(0, 10), inplace=True)

    # Evaluate date information
    day_info = []
    month_info = []
    year_info = []
    run_info = []
    hour_info = []
    current_day = None
    current_month = None
    current_year = None
    current_run = None
    current_hour = None
    for index, row in data.iterrows():
        day_info.append(current_day)
        month_info.append(current_month)
        year_info.append(current_year)
        run_info.append(current_run)
        hour_info.append(current_hour)
        if row.iloc[1] in months:
            current_day = int(row.iloc[0])
            current_month = row.iloc[1]
            current_year = int(row.iloc[2])
            current_run = int(row.iloc[3].strip().replace("Run ", ""))
            current_hour = f"{row.iloc[4][0:2]}:{row.iloc[4][2:4]}"

    # Change column names
    data.columns = ['post', 'degrees', 'azimuth_degrees', 'wind_direction', '50', '100', '200']

    # Add date info to the dataframe
    data['run'] = run_info
    # Convert to pandas Timestamp
    timestamps = [pd.Timestamp(year=y, month=months.index(m)+1, day=d, hour=int(h.split(':')[0]), minute=int(h.split(':')[1])) if y is not None else None for y, m, d, h in zip(year_info, month_info, day_info, hour_info)]
    data['timestamp'] = timestamps

    # Change data types
    data['post'] = pd.to_numeric(data['post'], errors='coerce')
    data['post'] = data['post'].astype(int, errors='ignore')

    data['degrees'] = pd.to_numeric(data['degrees'], errors='coerce')
    data['degrees'] = data['degrees'].astype(int, errors='ignore')

    data['azimuth_degrees'] = pd.to_numeric(data['azimuth_degrees'], errors='coerce')
    data['azimuth_degrees'] = data['azimuth_degrees'].astype(int, errors='ignore')

    data['wind_direction'] = pd.to_numeric(data['wind_direction'], errors='coerce')
    data['50'] = pd.to_numeric(data['50'], errors='coerce')
    data['100'] = pd.to_numeric(data['100'], errors='coerce')
    data['200'] = pd.to_numeric(data['200'], errors='coerce')

    # Drop NaN values
    to_drop = data[data['post'].isna() | data['degrees'].isna()].index
    data.drop(to_drop, inplace=True)

    # Reset index
    data.reset_index(drop=True, inplace=True)

    return data


In [445]:
def add_additional_data(data: pd.DataFrame, meteo_data: pd.DataFrame, columns = None):
    # Drop rows until relevant data is reached
    i = 0
    for index, row in meteo_data.iterrows():
        if str(row.iloc[1]).strip().lower() == "run":
            i = index
            break
    meteo_data.drop(range(0, i), inplace=True)
    meteo_data.reset_index(drop=True, inplace=True)
    # Check if data type row exists
    data_type_exists = False
    if str(meteo_data.iloc[1, 1]).strip().lower() == 'number':
        data_type_exists = True
    # Drop first column
    meteo_data.drop(columns = [meteo_data.columns[0]], inplace=True)

    # Add meteo data as new columns
    meteo_info = []
    for index, row in data.iterrows():
        run = row['run']
        meteo_info.append(meteo_data.iloc[int(run) if not data_type_exists else int(run) + 1])

    # Convert obtained values to a pandas DataFrame
    meteo_columns = pd.DataFrame(data=meteo_info)
    meteo_columns.columns = [c.strip() for c in meteo_data.iloc[0]] if columns is None else columns

    # Drop run column, if present
    run_column_names = ['Run', 'run']
    for run_column_name in run_column_names:
        if run_column_name in meteo_columns.columns:
            meteo_columns.drop(columns=[run_column_name], inplace=True)

    # Concat the two dataframes
    meteo_columns.reset_index(inplace=True, drop=True)
    return pd.concat([data, meteo_columns], axis='columns')



In [447]:
 data1 = pd.read_excel("RHILL1Update01.xls", sheet_name=2)
data1_meteo = pd.read_excel("RHILL1Update01.xls", sheet_name=1)
data1_correction = pd.read_excel("RHILL1Update01.xls", sheet_name=0)

data2 = pd.read_excel("RHILL2Update01.xls", sheet_name=2)
data2_meteo = pd.read_excel("RHILL2Update01.xls", sheet_name=1)
data2_correction = pd.read_excel("RHILL2Update01.xls", sheet_name=0)

parsed1 = add_additional_data(parse_measure_data(data1), data1_meteo)
parsed1 = add_additional_data(parsed1, data1_correction, ['Run', 'Q', '50 Correction', '100 Correction', '200 Correction'])

parsed2 = parse_measure_data(data2.iloc[:, 0:7])
parsed2 = add_additional_data(parsed2, data2_meteo)
parsed2 = add_additional_data(parsed2, data2_correction)

In [444]:
parsed1

,post,degrees,azimuth_degrees,wind_direction,50,100,200,run,timestamp,U2,...,U6.0,U12.0,T1.5,T3.0,T6.0,T12.0,Q,50 Correction,100 Correction,200 Correction
0,7.0,18.0,43.0,NaN,NaN,NaN,NaN,1.0,1954-08-13 09:15:00,2.17,...,-99.00,-99.00,-99.0,-99.00,-99.00,-99.00,1.00,0.93,0.93,0.92
1,8.0,21.0,46.0,NaN,NaN,NaN,NaN,1.0,1954-08-13 09:15:00,2.17,...,-99.00,-99.00,-99.0,-99.00,-99.00,-99.00,1.00,0.93,0.93,0.92
2,9.0,24.0,49.0,NaN,NaN,NaN,0.010,1.0,1954-08-13 09:15:00,2.17,...,-99.00,-99.00,-99.0,-99.00,-99.00,-99.00,1.00,0.93,0.93,0.92
3,10.0,27.0,52.0,0.42,NaN,NaN,0.015,1.0,1954-08-13 09:15:00,2.17,...,-99.00,-99.00,-99.0,-99.00,-99.00,-99.00,1.00,0.93,0.93,0.92
4,11.0,30.0,55.0,NaN,NaN,NaN,0.005,1.0,1954-08-13 09:15:00,2.17,...,-99.00,-99.00,-99.0,-99.00,-99.00,-99.00,1.00,0.93,0.93,0.92
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
898,27.0,78.0,103.0,4.32,3.330,0.38,0.035,29.0,1955-11-08 23:10:00,1.78,...,2.62,3.11,2.6,3.02,3.29,3.44,7.96,0.96,0.96,0.97
899,28.0,81.0,106.0,0.66,0.710,0.07,NaN,29.0,1955-11-08 23:10:00,1.78,...,2.62,3.11,2.6,3.02,3.29,3.44,7.96,0.96,0.96,0.97
900,29.0,84.0,109.0,0.17,0.015,NaN,NaN,29.0,1955-11-08 23:10:00,1.78,...,2.62,3.11,2.6,3.02,3.29,3.44,7.96,0.96,0.96,0.97
901,30.0,87.0,112.0,0.17,NaN,NaN,NaN,29.0,1955-11-08 23:10:00,1.78,...,2.62,3.11,2.6,3.02,3.29,3.44,7.96,0.96,0.96,0.97


In [448]:
parsed2

,post,degrees,azimuth_degrees,wind_direction,50,100,200,run,timestamp,U(m/s),...,T1.5,T3.0,T6.0,T12.0,10-min,3-min,0.5-min,10-min,3-min,0.5-min
0,37.0,1.5,76.5,NaN,NaN,NaN,NaN,1.0,1957-09-24 19:35:00,2.14,...,12.12,12.51,12.73,12.94,45.1,45.1,45.1,0.90,0.92,0.83
1,38.0,1.5,78.0,0.42,NaN,NaN,NaN,1.0,1957-09-24 19:35:00,2.14,...,12.12,12.51,12.73,12.94,45.1,45.1,45.1,0.90,0.92,0.83
2,39.0,1.5,79.5,NaN,NaN,NaN,NaN,1.0,1957-09-24 19:35:00,2.14,...,12.12,12.51,12.73,12.94,45.1,45.1,45.1,0.90,0.92,0.83
3,40.0,1.5,81.0,NaN,NaN,NaN,NaN,1.0,1957-09-24 19:35:00,2.14,...,12.12,12.51,12.73,12.94,45.1,45.1,45.1,0.90,0.92,0.83
4,41.0,1.5,82.5,NaN,NaN,NaN,NaN,1.0,1957-09-24 19:35:00,2.14,...,12.12,12.51,12.73,12.94,45.1,45.1,45.1,0.90,0.92,0.83
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
562,75.0,1.5,133.5,NaN,NaN,NaN,NaN,10.0,1957-12-03 11:20:00,4.07,...,3.75,3.37,3.16,2.93,93.4,97.5,97.5,0.94,0.96,0.97
563,76.0,1.5,135.0,NaN,NaN,NaN,NaN,10.0,1957-12-03 11:20:00,4.07,...,3.75,3.37,3.16,2.93,93.4,97.5,97.5,0.94,0.96,0.97
564,77.0,1.5,136.5,NaN,NaN,NaN,NaN,10.0,1957-12-03 11:20:00,4.07,...,3.75,3.37,3.16,2.93,93.4,97.5,97.5,0.94,0.96,0.97
565,78.0,1.5,138.0,0.42,NaN,NaN,NaN,10.0,1957-12-03 11:20:00,4.07,...,3.75,3.37,3.16,2.93,93.4,97.5,97.5,0.94,0.96,0.97
